In [5]:
import pandas as pd

ratings1 = pd.read_csv('data/ratings1.csv')
ratings2 = pd.read_csv('data/ratings2.csv')
dates = pd.read_csv('data/dates.csv')

Основные функции concat:
* objs — список объектов DataFrame ([df1, df2,…]), которые должны быть сконкатенированы;
* axis — ось определяет направление конкатенации: 0 — конкатенация по строкам (по умолчанию), 1 — конкатенация по столбцам;
* join — либо inner (пересечение), либо outer (объединение); рассмотрим этот момент немного позже;
* ignore_index — по умолчанию установлено значение False, которое позволяет значениям индекса оставаться такими, какими они были в исходных данных. Если установлено значение True, параметр будет игнорировать исходные значения и повторно назначать значения индекса в последовательном порядке.

Для корректной конкатенации по строкам объединяемые таблицы должны иметь одинаковую структуру — идентичное число и имена столбцов.

In [2]:
ratings = pd.concat([ratings1, ratings2])
display(ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
60831,610,166534,4.0
60832,610,168248,5.0
60833,610,168250,5.0
60834,610,168252,5.0


В результате мы увеличили первую таблицу, добавив снизу строки второй таблицы. Но rows == 100837, а последний индекс 60835, нужно использовать ignore_index=True, для создания новых индексов, а не копирования существующих.

In [3]:
ratings = pd.concat(
    [ratings1, ratings2],
    ignore_index=True
)
display(ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100832,610,166534,4.0
100833,610,168248,5.0
100834,610,168250,5.0
100835,610,168252,5.0


In [6]:
display('Число строк в таблице ratings: ', ratings.shape[0])
display('Число строк в таблице dates: ', dates.shape[0])
display(ratings.shape[0] == dates.shape[0])

'Число строк в таблице ratings: '

100837

'Число строк в таблице dates: '

100836

False

Количество строк различное, не получится произвести корректную вертикальную конкатенацию 

На самом деле очень просто: при выгрузке данных информация об оценках какого-то  пользователя попала в обе таблицы (ratings1 и ratings2). В результате конкатенации случилось дублирование строк. В данном примере их легко найти — выведем последнюю строку таблицы ratings1 и первую строку таблицы ratings2:

In [7]:
display(ratings1.tail(1))
display(ratings2.head(1))

,userId,movieId,rating
40000,274,5621,2.0


,userId,movieId,rating
0,274,5621,2.0


Чтобы очистить таблицу от дублей, мы можем воспользоваться методом DataFrame drop_duplicates(), который удаляет повторяющиеся строки в таблице. Не забываем обновить индексы после удаления дублей, выставив параметр ignore_index в методе drop_duplicates() на значение True:

In [8]:
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])

Число строк в таблице ratings:  100836


конкатенируем dates и ratings по столбам

In [9]:
ratings_dates = pd.concat([ratings, dates], axis=1)
display(ratings_dates.tail(7))

,userId,movieId,rating,date
100829,610,164179,5.0,2017-05-03 21:07:11
100830,610,166528,4.0,2017-05-04 06:29:25
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12
100835,610,170875,3.0,2017-05-03 21:20:15


6.3 Допустим, в ваше распоряжение предоставлена директория "./Root/users/". В данной директории содержатся csv-файлы, в каждом из которых хранится информация об идентификаторах пользователей (user_id) и ссылки на их фотографии (photo_url). Каждый файл из папки users имеет примерно следующую структуру:

*   user_id     image_url
* 0 id001 http://example.com/img/id001.png
* 0 id002 http://example.com/img/id001.png
...

При проверке в директории может быть сколько угодно файлов (директория может изменяться в зависимости от устройства файловой системы).

Вам необходимо написать функцию concat_user_files(path), параметром которой является path — путь до директории. Функция должна объединить информацию из предоставленных вам файлов в один DataFrame и вернуть его. 

Список названий всех файлов, находящихся в директории, вы можете получить с помощью функции os.listdir(path) из модуля os (модуль уже импортирован в файле main.py). Например, для директории "./Root/users/" результатом работы функции будет список:

print(os.listdir('./Root/users/'))
['users2.csv', 'users1.csv', 'users3.csv']
Примечание. Модуль os позволяет работать с операционной системой компьютера прямо из кода. Подробнее о нем вы можете почитать здесь.

Отсортируйте этот список, прежде чем производить объединение файлов.

Когда вы получите отсортированный список, вам останется только прочитать все csv-файлы из списка в цикле и объединить прочитанные таблицы между собой.

Однако обратите внимание, что метод os.listdir() возвращает только список имён файлов в указанной директории, а при чтении файла необходимо указывать полный путь до него. То есть путь для чтения будет таким:

'./Root/users/{file_name}'
Не забудьте обновить индексы результирующей таблицы после объединения.

Учтите, что на тестовом наборе файлов в результате объединения могут возникнуть дубликаты, от которых необходимо будет избавиться.

Например, для директории "./Root/users/" результирующая таблица должна иметь следующий вид:

*   user_id     image_url
* 0 id001 http://example.com/img/id001.png
* 0 id002 http://example.com/img/id001.png
...


In [15]:
import os

#users1 = pd.read_csv('data/users1.csv')
#users2 = pd.read_csv('data/users2.csv')
#users3 = pd.read_csv('data/users3.csv')
#users4 = pd.read_csv('data/users4.csv')

def concat_user_files(path):
    data = pd.DataFrame()
    list_1 = list(os.listdir(path))
    list_1.sort()
    for file in list_1:
        file_csv = pd.read_csv(path + '/' + file)
        data = pd.concat([data, file_csv], ignore_index=True)
    data = data.drop_duplicates(ignore_index=True)
    return data
    
display(concat_user_files('data/users'))

#display(concat_user_files(path = 'users2')) ? Как найдёт путь?

,user_id,image_url
0,id001,http://example.com/img/id001.png
1,id002,http://example.com/img/id002.png
2,id003,http://example.com/img/id003.png
3,id004,http://example.com/img/id004.png
4,id005,http://example.com/img/id005.png
5,id006,http://example.com/img/id006.png
6,id007,http://example.com/img/id007.png
7,id008,http://example.com/img/id008.png
8,id009,http://example.com/img/id009.png
9,id0010,http://example.com/img/id0010.png
